In [1]:
import geopandas as gpd
import pandas as pd

In [3]:
df_patch = pd.read_parquet('burst_patch_table.parquet')
df_patch.shape

(384579, 3)

In [4]:
df_rtc = pd.read_parquet('rtc_s1_table.parquet')
df_rtc.shape

(237853, 8)

In [21]:
df_patch_count_by_burst = df_patch.groupby('jpl_burst_id').size().reset_index(drop=False).rename(columns={0: 'patch_count_per_burst'})
df_patch_count_by_burst.head()

,jpl_burst_id,patch_count_per_burst
0,T005-008688-IW2,144
1,T005-008688-IW3,122
2,T005-008689-IW1,133
3,T005-008689-IW2,143
4,T005-008689-IW3,122


In [22]:
df_date_count_by_burst = df_rtc.groupby('jpl_burst_id').size().reset_index(drop=False).rename(columns={0: 'date_count_per_burst'})
df_date_count_by_burst.head()

,jpl_burst_id,date_count_per_burst
0,T005-008688-IW2,66
1,T005-008688-IW3,66
2,T005-008689-IW1,66
3,T005-008689-IW2,66
4,T005-008689-IW3,66


In [23]:
df_count_rtc = pd.merge(df_date_count_by_burst, df_patch_count_by_burst, on='jpl_burst_id', how='inner')
df_count_rtc.head()

,jpl_burst_id,date_count_per_burst,patch_count_per_burst
0,T005-008688-IW2,66,144
1,T005-008688-IW3,66,122
2,T005-008689-IW1,66,133
3,T005-008689-IW2,66,143
4,T005-008689-IW3,66,122


In [24]:
df_count_rtc['total_samples_per_burst'] = df_count_rtc['date_count_per_burst'] * df_count_rtc['patch_count_per_burst']
df_count_rtc.head()

,jpl_burst_id,date_count_per_burst,patch_count_per_burst,total_samples_per_burst
0,T005-008688-IW2,66,144,9504
1,T005-008688-IW3,66,122,8052
2,T005-008689-IW1,66,133,8778
3,T005-008689-IW2,66,143,9438
4,T005-008689-IW3,66,122,8052


In [26]:
df_count_rtc['total_samples_cumul'] = df_count_rtc.total_samples_per_burst.cumsum()
df_count_rtc.head()

,jpl_burst_id,date_count_per_burst,patch_count_per_burst,total_samples_per_burst,total_samples_cumul
0,T005-008688-IW2,66,144,9504,9504
1,T005-008688-IW3,66,122,8052,17556
2,T005-008689-IW1,66,133,8778,26334
3,T005-008689-IW2,66,143,9438,35772
4,T005-008689-IW3,66,122,8052,43824


In [29]:
df_count_rtc.total_samples_cumul.iloc[-1], df_count_rtc.total_samples_per_burst.sum() 

(27234711, 27234711)

In [19]:
df_count_rtc.to_parquet('rtc_spatiotemporal_samples.parquet')